In [ ]:
library(httr)
library(sf)
library(dplyr)

# Load environment variables

source('config.R') # ATLAS_API_HOST, ATLAS_API_TOKEN

In [ ]:
# Function arguments
taxa_id <- 32
years <- 2013

In [ ]:
# PostgREST API Request
request <- paste0(ATLAS_API_HOST, '/observations',
                  '?select=id,datasets(id,original_source)',
                  ',taxa(id,scientific_name),variables(id,name,unit)',
                  ',year_obs,month_obs,day_obs,time_obs,obs_value,geom',
                  taxa_filter,years_filter, collapse='')

In [ ]:
# Update request from filters defined by arguments
if (! is.null(taxa_id)){
    request <- paste0(request, '&id_taxa=in.(', paste0(taxa_id, collapse=','),')', collapse='')
}

if (! is.null(years)){
    request <- paste0(request, '&year_obs=in.(', paste0(years, collapse=','),')', collapse='')
}

In [ ]:
# Make request and parse response
response <- GET(URLencode(request),
                add_headers(Authorization = paste('Bearer', ATLAS_API_TOKEN, sep = ' ')))

stopifnot(status_code(response) == 200)
out <- bind_rows(lapply(content(response, as = 'parsed', type = 'application/json'), unlist))

colnames(out) <- sub('\\.','_',names(out))

In [ ]:
# Convert geometry wkb values to sf
to_sf <- function(geom_char){st_as_sfc(structure(geom_char, class = 'WKB'), EWKB = TRUE)}
out <- mutate(out, geom = to_sf(geom))